# Add Image Metrics to an existing Table

In [ ]:
import umap

In [ ]:
from pathlib import Path

DATA_PATH = Path("../data/coco128/images").absolute().as_posix()


## Write the initial table

In [ ]:
import os
import tlc

table_writer = tlc.TableWriter(
    table_name="coco128",
    dataset_name="coco128",
    project_name="add-image-metrics",
    description="COCO128 dataset",
    column_schemas={
        "image": tlc.ImagePath
    }
)

for image_name in os.listdir(DATA_PATH):
    image_path = os.path.join(DATA_PATH, image_name)
    table_writer.add_row({"image": image_path})

table = table_writer.finalize()


## Extend the table with embeddings from a pre-trained model

In [ ]:
from transformers import ViTFeatureExtractor, ViTModel
from PIL import Image
import torch
import requests
from torchvision import transforms

# Load the model and feature extractor
model_name = "google/vit-base-patch16-224"
feature_extractor = ViTFeatureExtractor.from_pretrained(model_name)
model = ViTModel.from_pretrained(model_name)

# Define the transformation pipeline
preprocess = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=feature_extractor.image_mean, std=feature_extractor.image_std),
])

# Load a batch of images (example with three images)
image_urls = [
    table[0]["image"],
    table[1]["image"],
]

# Preprocess the images
images = []
for url in image_urls:
    image = Image.open(url).convert("RGB")
    image = preprocess(image)
    images.append(image)

# Create a batch tensor
batch = torch.stack(images)

# Pass the batch through the model to get embeddings
with torch.no_grad():
    outputs = model(batch)
    embeddings = outputs.last_hidden_state[:, 0, :]  # Extract embeddings for [CLS] token

print("Embeddings shape:", embeddings.shape)  # (batch_size, hidden_size)
print("Embeddings:", embeddings)


In [ ]:
import tqdm

extended_table_writer = tlc.TableWriter(
    table_name="added-embeddings",
    dataset_name="coco128",
    project_name="add-embeddings",
    description="COCO128 dataset with added embeddings",
    column_schemas={
        "image": tlc.ImagePath,
        "embedding": tlc.Schema(
            value=tlc.Float32Value(number_role=tlc.NUMBER_ROLE_NN_EMBEDDING),
            writable=False,
            size0=tlc.DimensionNumericValue(768),
            sample_type="hidden",
        ),
    },
)

for row in tqdm.tqdm(table):
    image_path = row["image"]
    image = Image.open(image_path).convert("RGB")
    image = preprocess(image)
    image = image.unsqueeze(0)
    with torch.no_grad():
        outputs = model(image)
        embedding = outputs.last_hidden_state[:, 0, :].squeeze().numpy()
    new_row = {**row, "embedding": embedding.tolist()}
    extended_table_writer.add_row(new_row)

extended_table = extended_table_writer.finalize()

print(extended_table[0])

In [ ]:
reducer = tlc.create_reducer("umap", {"n_components": 2, "metric": "euclidean"})
reduced_table_url = reducer.fit_reduction_method(extended_table, "embedding")

In [ ]:
reduced_table = tlc.Table.from_url(reduced_table_url)
print(reduced_table[0])